# Assignment 3: Summarization Tests

**Description:** This assignment covers summarization outputs. You will compare three different types of solutions, all using an encoder decoder architecture. You should also be able to develop an intuition for:


* How well summarization systems work
* The effects of using different pre-training and fine-tuning checkpoints on outcomes
* The effects of hyperparameters on outcomes
* Evaluation of output using ROUGE



This notebook should be run on a Google Colab but it does not require a GPU. By default, when you open the notebook in Colab it will NOT configure a GPU.  Summarization commands can take up to five minutes to run depending on the hyperparameters you use. This notebook will NOT run on your GCP instance as the summary models are larger than the avaialble memory.


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/datasci-w266/2023-summer-main/blob/master/assignment/a3/Summarization_test.ipynb)

The overall assignment structure is as follows:

 Setup

1. T5 for generic summarization

2. Pegasus for headline summarization

3. Pegasus for longer generation




**INSTRUCTIONS:**:

* Questions are always indicated as **QUESTION:**, so you can search for this string to make sure you answered all of the questions. You are expected to fill out, run, and submit this notebook, as well as to answer the questions in the **answers** file as you did in a1 and a2.

* **### YOUR CODE HERE** indicates that you are supposed to write code.




## Setup

In [1]:
!pip install -q sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.0 MB/s eta 0:00:00


In [2]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 86.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.7 MB/s eta 0:00:00


In [3]:
!pip install -q evaluate
import evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.6 MB/s eta 0:00:00


In [4]:
!pip install -q rouge_score

  Preparing metadata (setup.py) ... done


In [5]:
#let's make longer output readable without horizontal scrolling
from pprint import pprint

Let's leverage the pre-trained and fine tuned models on HuggingFace to demonstrate some capabilities with abstractive summarization and language generation.  They include models/checkpoints that were fine tuned on a particular dataset.  In our case we'll focus on one dataset that emphasizes a one line output and another that emphasizes a multi-line output.

We'll use this same toy article as the input to all of our summarization attempts.  That way we have the ability to compare. We'll also create two references for evaluation.  These are the targets you are trying to meet.  One reference is for the longer output. The second reference is the short one for the one line output.

In [6]:

ARTICLE_TO_SUMMARIZE = (
    "Nearly 800 thousand customers are scheduled to be affected by the shutoffs which are expected to last through at least midday tomorrow. "
    "PG&E stated it scheduled the blackouts in response to forecasts for high winds amid dry conditions. "
    "The aim is to reduce the risk of wildfires. "
    "If Pacific Gas & Electric Co, a unit of PG&E Corp, goes through with another public safety power shutoff, "
    " it would be the fourth round of mass blackouts imposed by the utility since Oct. 9, when some 730,000 customers were left in the dark. "
    "The recent wave of precautionary shutoffs have drawn sharp criticism from Governor Gavin Newsom, state regulators and consumer activists as being overly broad in scale."
    "Newsom blames PG&E for doing too little to properly maintain and secure its power lines against wind damage."
    "Utility executives have acknowledged room for improvement while defending the sprawling scope of the power cutoffs as a matter of public safety."
    "The record breaking drought has made the current conditions even worse than in previous years. "
    "It exponentially increases the probability of large scale wildfires. "
)

LONG_REFERENCE = (
    "Many PG&E customers could be affected by public safety power shutoffs in response to forecasts for high winds and dry conditions. "
    "The record breaking drought exponentially increases the probability of large scale wildfires. "
    "Despite being criticized by Governor Newsom for being overly broad, company officials defend the cutoffs as a matter of public safety. "
)

SHORT_REFERENCE = (
    "California's largest utility is set to turn off power to hundreds of thousands of customers in an effort to reduce the risk of wildfires. "
)

How long is our article to summarize?  Obviously our summary should be shorter since it is supposed to be "abridged."

In [7]:
len(str.split(ARTICLE_TO_SUMMARIZE))

177

In [25]:
len(str.split(SHORT_REFERENCE))

24

In [81]:
len(str.split(LONG_REFERENCE))

54

## 1. T5 for Generic Summarization

T5 is an encoder decoder architecture that has been trained on multiple tasks, so not purely summarization.  You can read more about it [here](https://huggingface.co/docs/transformers/model_doc/t5).

In [8]:
from transformers import T5Tokenizer, TFT5ForConditionalGeneration

t5model = TFT5ForConditionalGeneration.from_pretrained("t5-base")
t5tokenizer = T5Tokenizer.from_pretrained("t5-base")

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:238: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the

In [9]:
t5model.summary()

Model: "tft5_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (Embedding)          multiple                  24674304  
                                                                 
 encoder (TFT5MainLayer)     multiple                  109628544 
                                                                 
 decoder (TFT5MainLayer)     multiple                  137949312 
                                                                 
Total params: 222903552 (850.31 MB)
Trainable params: 222903552 (850.31 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Since T5 can perform multiple tasks we need to tell it what kind of output we want.  Therefore we need to prepend a "prompt" to our article text to make sure it does the right thing.

In [10]:
PROMPT = 'summarize: '
T5ARTICLE_TO_SUMMARIZE = PROMPT + ARTICLE_TO_SUMMARIZE

In [11]:
inputs = t5tokenizer(T5ARTICLE_TO_SUMMARIZE, max_length=1024, truncation=True, return_tensors="tf")

What do the inputs look like?  How does it compare with what we've seen from BERT?

In [17]:
inputs

{'input_ids': <tf.Tensor: shape=(1, 241), dtype=int32, numpy=
array([[21603,    10, 10455,   120,  8640,  7863,   722,    33,  5018,
           12,    36,  4161,    57,     8,  6979,  1647,     7,    84,
           33,  1644,    12,   336,   190,    44,   709,  2076,  1135,
         5721,     5,     3,  7861,   184,   427,  4568,    34,  5018,
            8,  1001,   670,     7,    16,  1773,    12,  7555,     7,
           21,   306, 13551, 18905,  2192,  1124,     5,    37,  2674,
           19,    12,  1428,     8,  1020,    13,  3645,  6608,     7,
            5,   156,  5824,  6435,     3,   184,  8666,   638,     6,
            3,     9,  1745,    13,     3,  7861,   184,   427, 10052,
            6,  1550,   190,    28,   430,   452,  1455,   579,  6979,
         1647,     6,    34,   133,    36,     8,  4509,  1751,    13,
         3294,  1001,   670,     7,     3, 16068,    57,     8,  6637,
          437,  6416,     5,  9902,   116,   128,   489, 17093,   722,
          130, 

Let's just run T5 using it's default hyperparameters and see what happens.  We'll hold on to the output in the candidate variable.  What do you think about the output?

In [13]:
# Generate Summary
summary_ids = t5model.generate(inputs["input_ids"],
                               max_new_tokens=30
)
candidate = t5tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
pprint(candidate[0], compact=True)

('PG&e shuts off power to 800 thousand customers . the shutoffs are scheduled '
 'to last through at least midday tomorrow .')


### 1.a Checkpoint Configuration

We're using the `t5-base` configuration and we know we can run out of the box to do summarization which means it has some hyperparameters set as defaults.  These may or may not be what we want to use.  How do we know which values are set as defaults?

HuggingFace provides access to the default hyperparameters via the AutoConfig object which we call below.  We simply pass in the name of the checkpoint we're using -- `t5-base` in this case.


In [14]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained("t5-base")

config

T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_stopping": true,
      "max_length": 300,
   

Look at the `task_specific_params` for summarization. You can see that this `t5-base` checkpoint has some values such as min_length and max_length as well as no_repeat_ngram_size and num_beams.  You can affect the size and content of the output by modifying these parameters which you will do below.

You can also look at the full set of possible parameters in the [TFGenerationMixin](https://huggingface.co/docs/transformers/v4.18.0/en/main_classes/text_generation#transformers.generation_tf_utils.TFGenerationMixin) class available to all of the pre-trained models.

HuggingFace has also written [a very helpful blog post](https://huggingface.co/blog/how-to-generate) that explains and discusses various strategies for text generation and how to manipulate the hyperparameters.  They discuss the two approaches of beam search (which we have discussed in the async and live session) as well as sampling (which tries to randomly pick the next word within a k-sized distribution of highly probable choices).

**Please read the blog post before you proceed.**

For your reference, here's a more complex, technical, and thorough [HuggingFace guide](https://huggingface.co/docs/transformers/main/en/generation_strategies) for controlling generation of text.  The blog post above is all you need to read to complete the assignment.

### 1.b ROUGE for summarization evaluation

ROUGE is the metric that has been traditionally used to evaluate sumarization results.  The ROUGE metric expects a reference as input and it will evaluate a candidate against that reference.  ROUGE-1 calculates the number of words in the reference that occur in the candidate.  ROUGE-2 performs that same calculation but for bigrams in the reference. ROUGE-L calculates the longest common subsequence of reference words that occur in the candidate.

HuggingFace provides a wrapper around [a library](https://huggingface.co/spaces/evaluate-metric/rouge) to calculate ROUGE metrics which you will use below.  Let's calculate the ROUGE score for the candidate you produced above.

In [18]:
rouge = evaluate.load('rouge')
predictions = candidate
references = [SHORT_REFERENCE]
results = rouge.compute(predictions=predictions,
                        references=references)
print(results)

{'rouge1': 0.2666666666666666, 'rouge2': 0.0930232558139535, 'rougeL': 0.22222222222222224, 'rougeLsum': 0.22222222222222224}


Let's experiment with the hyperparameters shown above.  Please experiment in the cell below.  The `num_beams` value is like a beam search.  It indicates the number of tries the model makes before showing you its best output.  The `no_repeat_ngram_size` is designed to help reduce repetition in the output.  `min_length` and `max_length` set boundaries on the size of the summary. You are free to use other hyperparameters as described in the [blog post](https://huggingface.co/blog/how-to-generate).

*There is no one correct answer to these questions.  There are ranges that tend to work better than others.  The goal is to have you experiment to help build intuition.  Please enter the values that you think are generating the most readable output.*

*Your readable output should consist of at least one complete sentence but does not have to end with a period and you must also have a ROUGE-1 score above 0.30 and ROUGE-L score equal to or above 0.25 when compared with the short reference.*

You can use the two cells below to come up with your answer.

In [48]:
# Generate Summary
# max_new_tokens = max_length in this case
# ARTICLE_TO_SUMMARIZE is 177 words long, and SHORT_REFERENCE is 24 words long.
summary_ids = t5model.generate(inputs["input_ids"],
### YOUR CODE HERE
                              early_stopping=True,
                              length_penalty=2.0,
                              max_length=80,
                              min_length=60,
                              no_repeat_ngram_size=3,
                              num_beams=4

### END YOUR CODE
)

candidate = t5tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
pprint(candidate[0], compact=True)

('PG&e shuts off power to 800 thousand customers in response to forecasts for '
 'high winds . the aim is to reduce the risk of wildfires . it would be the '
 'fourth round of mass blackouts imposed by the utility since october . a '
 'drought has made the current conditions even worse than in previous years .')


In [49]:
predictions = candidate
references = [SHORT_REFERENCE]
results = rouge.compute(predictions=predictions,
                        references=references)
print(results)

{'rouge1': 0.3846153846153846, 'rouge2': 0.21052631578947367, 'rougeL': 0.28205128205128205, 'rougeLsum': 0.28205128205128205}


In [25]:
""" records
default setting:
                              early_stopping=True,
                              length_penalty=2.0,
                              max_length=177,
                              min_length=30,
                              no_repeat_ngram_size=2,
                              num_beams=4
score: {'rouge1': 0.37499999999999994, 'rouge2': 0.20512820512820512, 'rougeL': 0.275, 'rougeLsum': 0.275}
==========================================================================================================================
                              early_stopping=True,
                              length_penalty=2.0,
                              max_length=177,
                              min_length=30,
                              no_repeat_ngram_size=2,
                              num_beams=5
score:{'rouge1': 0.1846153846153846, 'rouge2': 0.0, 'rougeL': 0.12307692307692307, 'rougeLsum': 0.12307692307692307}
==========================================================================================================================
                              early_stopping=True,
                              length_penalty=2.0,
                              max_length=177,
                              min_length=30,
                              no_repeat_ngram_size=2,
                              num_beams=7
score:  {'rouge1': 0.2926829268292683, 'rouge2': 0.125, 'rougeL': 0.17073170731707316, 'rougeLsum': 0.17073170731707316}
==========================================================================================================================
                              early_stopping=True,
                              length_penalty=2.0,
                              max_length=177,
                              min_length=30,
                              no_repeat_ngram_size=2,
                              num_beams=2
score:  {'rouge1': 0.21333333333333335, 'rouge2': 0.054794520547945195, 'rougeL': 0.16, 'rougeLsum': 0.16}
==========================================================================================================================
                              early_stopping=True,
                              length_penalty=2.0,
                              max_length=177,
                              min_length=30,
                              no_repeat_ngram_size=3,
                              num_beams=4
score:  {'rouge1': 0.4545454545454546, 'rouge2': 0.25, 'rougeL': 0.33333333333333337, 'rougeLsum': 0.33333333333333337}
==========================================================================================================================
                              early_stopping=True,
                              length_penalty=2.0,
                              max_length=177,
                              min_length=30,
                              no_repeat_ngram_size=4,
                              num_beams=4
score:  {'rouge1': 0.3582089552238806, 'rouge2': 0.1846153846153846, 'rougeL': 0.26865671641791045, 'rougeLsum': 0.26865671641791045}
==========================================================================================================================
                              early_stopping=True,
                              length_penalty=2.0,
                              max_length=177,
                              min_length=50,
                              no_repeat_ngram_size=3,
                              num_beams=4
score:  {'rouge1': 0.4615384615384615, 'rouge2': 0.25396825396825395, 'rougeL': 0.3384615384615385, 'rougeLsum': 0.3384615384615385}
==========================================================================================================================
                              early_stopping=True,
                              length_penalty=1.0,
                              max_length=177,
                              min_length=50,
                              no_repeat_ngram_size=3,
                              num_beams=4
score:  {'rouge1': 0.4615384615384615, 'rouge2': 0.25396825396825395, 'rougeL': 0.3384615384615385, 'rougeLsum': 0.3384615384615385}
==========================================================================================================================
                              early_stopping=True,
                              length_penalty=2.0,
                              max_length=177,
                              min_length=80,
                              no_repeat_ngram_size=3,
                              num_beams=4
score:  {'rouge1': 0.37037037037037035, 'rouge2': 0.20253164556962022, 'rougeL': 0.2716049382716049, 'rougeLsum': 0.2716049382716049}
==========================================================================================================================
                              early_stopping=True,
                              length_penalty=2.0,
                              max_length=80,
                              min_length=50,
                              no_repeat_ngram_size=3,
                              num_beams=4
score:  {'rouge1': 0.4615384615384615, 'rouge2': 0.25396825396825395, 'rougeL': 0.3384615384615385, 'rougeLsum': 0.3384615384615385}
==========================================================================================================================
                              early_stopping=True,
                              length_penalty=2.0,
                              max_length=80,
                              min_length=60,
                              no_repeat_ngram_size=3,
                              num_beams=4
score:  {'rouge1': 0.3846153846153846, 'rouge2': 0.21052631578947367, 'rougeL': 0.28205128205128205, 'rougeLsum': 0.28205128205128205}


Setting max_length is meaningless in this case, the longer the summary from 50 is, the lower the score gets.
"""

**QUESTION:**

1.1 What num_beams value gives you the most readable output that meets the score criteria?

1.2 Which no_repeat_ngram_size gives the most readable output that meets the score criteria?

1.3 What min_length value gives you the most readable output that meets the score criteria?

1.4 Which max_new_tokens value gives the most readable output that meets the score criteria?

1.5 What is the ROUGE-L score associated with your most readable candidate?

In [50]:
#In order to not consume all of the memory available in Colab we'll free up the memory we're using for these large language models
del t5model
del t5tokenizer


## 2. Pegasus for Headline Summarization

Pegasus is an encoder decoder architecture that has been explicitly pre-trained as an abstractive summarizer.  You can read more about it [here](https://huggingface.co/docs/transformers/model_doc/pegasus) and [here](https://arxiv.org/pdf/1912.08777.pdf).

We'll first use the `google/pegasus-xsum` checkpoint.  It is trained on a [summarization task](https://aclanthology.org/D18-1206.pdf) that reads a news article and then [emits a one line summary](https://huggingface.co/datasets/xsum).  This doesn't mean that it is limited in its output length.  It does mean that it works well with news article type inputs and tends toward shorter outputs.

In [51]:
from transformers import PegasusTokenizer, TFPegasusForConditionalGeneration

pmodel = TFPegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")
ptokenizer = PegasusTokenizer.from_pretrained("google/pegasus-xsum")

All model checkpoint layers were used when initializing TFPegasusForConditionalGeneration.

Some layers of TFPegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [52]:
pmodel.summary()

Model: "tf_pegasus_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (TFPegasusMainLayer)  multiple                  569748480 
                                                                 
 final_logits_bias (BiasLay  multiple                  96103     
 er)                                                             
                                                                 
Total params: 569844583 (2.12 GB)
Trainable params: 569748480 (2.12 GB)
Non-trainable params: 96103 (375.40 KB)
_________________________________________________________________


Let's see what kinds of default parameters are configured in to this checkpoint.

In [53]:
config = AutoConfig.from_pretrained("google/pegasus-xsum")

config

PegasusConfig {
  "_name_or_path": "google/pegasus-xsum",
  "activation_dropout": 0.1,
  "activation_function": "relu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "PegasusForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 16,
  "decoder_start_token_id": 0,
  "do_blenderbot_90_layernorm": false,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 16,
  "eos_token_id": 1,
  "extra_pos_embeddings": 0,
  "force_bos_token_to_be_generated": false,
  "forced_eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,


Generate the inputs using the pegasus tokenizer for this checkpoint.

In [54]:
inputs = ptokenizer(ARTICLE_TO_SUMMARIZE, max_length=1024, truncation=True, return_tensors="tf")

Let's get some output using just the default values and see what we're working with.

In [55]:
# Generate Summary
summary_ids = pmodel.generate(inputs["input_ids"]
)
pprint(ptokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0], compact=True)

("California's largest utility has announced plans to cut power to hundreds of "
 'thousands of customers in a bid to reduce the risk of wildfires.')


Let's experiment with the same set of hyperparameters for the Pegasus system.  It is designed for abstractive summarization. Remember that the checkpoint we are using was trained on data that generates a one line summary for the input article.

*Your readable output should consist of at least one complete sentence but does not have to end with a period and you must also have a ROUGE-1 score above 0.30 and ROUGE-L score equal to or above 0.25 when compared with the short reference.*

You can use the two cells below to experiment with hyperparameters and generating and scoring your outputs in order to answer questions 2.1 - 2.5 in your answers file.

**QUESTION:**

2.1 What num_beams value gives you the most readable output that meets the score criteria?

2.2 Which no_repeat_ngram_size gives the most readable output that meets the score criteria?

2.3 What min_length value gives you the most readable output that meets the score criteria?

2.4 Which max_new_tokens value gives the most readable output that meets the score criteria?

2.5 What is the ROUGE-L score associated with your most readable candidate?

In [75]:
# Generate Summary
summary_ids = pmodel.generate(inputs["input_ids"],
### YOUR CODE HERE
                              num_beams=4,
                              no_repeat_ngram_size=2,
                              min_length=20,
                              max_new_tokens=100
### END YOUR CODE
)
candidate = ptokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
pprint(candidate[0], compact=True)

("California's largest utility has announced plans to cut power to hundreds of "
 'thousands of customers in a bid to reduce the risk of wildfires.')


In [76]:
rouge = evaluate.load('rouge')
predictions = candidate
references = [SHORT_REFERENCE]
results = rouge.compute(predictions=predictions,
                        references=references)
print(results)

{'rouge1': 0.76, 'rouge2': 0.625, 'rougeL': 0.76, 'rougeLsum': 0.76}


In [ ]:
""" records
default setting:
                              num_beams=4,
                              no_repeat_ngram_size=3,
                              min_length=50,
                              max_new_tokens=100,

score: {'rouge1': 0.5588235294117647, 'rouge2': 0.45454545454545453, 'rougeL': 0.5588235294117647, 'rougeLsum': 0.5588235294117647}
==========================================================================================================================
                              num_beams=3,
                              no_repeat_ngram_size=3,
                              min_length=50,
                              max_new_tokens=100,

score: {'rouge1': 0.5294117647058824, 'rouge2': 0.45454545454545453, 'rougeL': 0.5294117647058824, 'rougeLsum': 0.5294117647058824}
==========================================================================================================================
                              num_beams=5,
                              no_repeat_ngram_size=3,
                              min_length=50,
                              max_new_tokens=100,

score: {'rouge1': 0.5588235294117647, 'rouge2': 0.45454545454545453, 'rougeL': 0.5588235294117647, 'rougeLsum': 0.5588235294117647}
==========================================================================================================================
                              num_beams=4,
                              no_repeat_ngram_size=4,
                              min_length=50,
                              max_new_tokens=100,

score: {'rouge1': 0.5588235294117647, 'rouge2': 0.45454545454545453, 'rougeL': 0.5588235294117647, 'rougeLsum': 0.5588235294117647}
==========================================================================================================================
                              num_beams=4,
                              no_repeat_ngram_size=2,
                              min_length=50,
                              max_new_tokens=100,

score: {'rouge1': 0.5, 'rouge2': 0.4054054054054054, 'rougeL': 0.5, 'rougeLsum': 0.5}
==========================================================================================================================
                              num_beams=4,
                              no_repeat_ngram_size=3,
                              min_length=60,
                              max_new_tokens=100,

score: {'rouge1': 0.46341463414634143, 'rouge2': 0.375, 'rougeL': 0.46341463414634143, 'rougeLsum': 0.46341463414634143}
==========================================================================================================================
                              num_beams=4,
                              no_repeat_ngram_size=3,
                              min_length=40,
                              max_new_tokens=100,

score: {'rouge1': 0.6031746031746031, 'rouge2': 0.4918032786885246, 'rougeL': 0.6031746031746031, 'rougeLsum': 0.6031746031746031}
==========================================================================================================================
                              num_beams=4,
                              no_repeat_ngram_size=3,
                              min_length=30,
                              max_new_tokens=100,

score: {'rouge1': 0.6440677966101696, 'rouge2': 0.5263157894736842, 'rougeL': 0.6440677966101696, 'rougeLsum': 0.6440677966101696}
==========================================================================================================================
                              num_beams=4,
                              no_repeat_ngram_size=3,
                              min_length=24,
                              max_new_tokens=100,

score: {'rouge1': 0.76, 'rouge2': 0.625, 'rougeL': 0.76, 'rougeLsum': 0.76}
==========================================================================================================================
if min_length = 20, score: {'rouge1': 0.76, 'rouge2': 0.625, 'rougeL': 0.76, 'rougeLsum': 0.76}

We are comparing with short reference, the shorter summary can be the better. max_length plays no effects here.
short reference is 24 words in length

Delete that Pegasus model and tokenizer so we can load the next one.

In [77]:
del pmodel
del ptokenizer

## 3. Pegasus for Longer Generation

Now let's try to produce a longer summary of our article.  In order to do that we are going to use a different fine-tuned checkpoint for Pegasus.  This checkpoint is fine-tuned on the [CNN/Daily Mail](https://huggingface.co/datasets/cnn_dailymail) set of news articles.  The references are on the order of several sentences long.

In [78]:
from transformers import PegasusTokenizer, TFPegasusForConditionalGeneration

cnnmodel = TFPegasusForConditionalGeneration.from_pretrained("google/pegasus-cnn_dailymail", from_pt=True)
cnntokenizer = PegasusTokenizer.from_pretrained("google/pegasus-cnn_dailymail", from_pt=True)

All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Let's see how this checkpoint is configured by default:

In [ ]:
config = AutoConfig.from_pretrained("google/pegasus-cnn_dailymail")

config

Let's tokenize our input for this checkpoint.

In [79]:
cnninputs = cnntokenizer(ARTICLE_TO_SUMMARIZE, max_length=1024, truncation=True, return_tensors="tf")

Run the summarizer with the defaults and let's see what it looks like.

In [80]:
# Generate Summary
summary_ids = cnnmodel.generate(inputs["input_ids"]
)

pprint(cnntokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0], compact=True)

('Nearly 800 thousand customers are scheduled to be affected by the shutoffs '
 'which are expected to last through at least midday tomorrow .<n>PG&E stated '
 'it scheduled the blackouts in response to forecasts for high winds amid dry '
 'conditions .<n>The aim is to reduce the risk of wildfires .')


Let's again experiment with the same set of hyperparameters (but possibly with different values) for the Pegasus system.  It is designed for abstractive summarization and this checkpoint is based on multi-line outputs.  We'll evaluate it against the long reference record.

*Your readable multi-line output must have a ROUGE-1 score above 0.25 and a ROUGE-L score above 0.15.*

You can use the two cells below to experiment with hyperparameters and generating and scoring your outputs in order to answer questions 3.1 - 3.5 in your answers file.

**QUESTION:**

3.1 What num_beams value gives you the most readable output that meets the score criteria?

3.2 Which no_repeat_ngram_size gives the most readable output that meets the score criteria?

3.3 What min_length value gives you the most readable output that meets the score criteria?

3.4 Which max_new_tokens value gives you the most readable output that meets the score criteria?

3.5 What is the ROUGE-L score associated with your most readable candidate?

In [97]:
# Generate Summary
summary_ids = cnnmodel.generate(cnninputs["input_ids"],
### YOUR CODE HERE
                              num_beams=3,
                              no_repeat_ngram_size=2,
                              min_length=80,
                              max_new_tokens=100

### END YOUR CODE
                             )
candidate = cnntokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
pprint(candidate[0], compact=True)

('The aim of the shutoffs is to reduce the risk of wildfires .<n>The record '
 'breaking drought has made the current conditions even worse than in previous '
 'years. It exponentially increases the probability of large scale '
 'wildfires.It also causes power outages when trees fall on power lines. PG&E '
 'says it will try to restore power as quickly as possible. The blackouts are '
 'expected to last through at least midday tomorrow.')


In [98]:
rouge = evaluate.load('rouge')
predictions = candidate
references = [LONG_REFERENCE]
results = rouge.compute(predictions=predictions,
                        references=references)
print(results)

{'rouge1': 0.3464566929133858, 'rouge2': 0.176, 'rougeL': 0.23622047244094488, 'rougeLsum': 0.23622047244094488}


In [ ]:
"""
LONG_REFERENCE is  54 in length
records:
                    num_beams=4,
                    no_repeat_ngram_size=2,
                    min_length=54,
                    max_new_tokens=100,

score: {'rouge1': 0.4038461538461538, 'rouge2': 0.2156862745098039, 'rougeL': 0.34615384615384615, 'rougeLsum': 0.34615384615384615}
==========================================================================================================================
                    num_beams=4,
                    no_repeat_ngram_size=2,
                    min_length=70,
                    max_new_tokens=100,

score: {'rouge1': 0.40336134453781514, 'rouge2': 0.2393162393162393, 'rougeL': 0.31932773109243695, 'rougeLsum': 0.31932773109243695}
==========================================================================================================================
********
                    num_beams=4,
                    no_repeat_ngram_size=2,
                    min_length=80,
                    max_new_tokens=100,

score: {'rouge1': 0.46875, 'rouge2': 0.3174603174603175, 'rougeL': 0.390625, 'rougeLsum': 0.390625}
==========================================================================================================================
                    num_beams=4,
                    no_repeat_ngram_size=2,
                    min_length=90,
                    max_new_tokens=100,

score: {'rouge1': 0.45925925925925926, 'rouge2': 0.3007518796992481, 'rougeL': 0.3703703703703703, 'rougeLsum': 0.3703703703703703}
==========================================================================================================================
                    num_beams=4,
                    no_repeat_ngram_size=3,
                    min_length=90,
                    max_new_tokens=100,

score: {'rouge1': 0.3857142857142857, 'rouge2': 0.17391304347826086, 'rougeL': 0.3142857142857143, 'rougeLsum': 0.3142857142857143}
==========================================================================================================================
                    num_beams=4,
                    no_repeat_ngram_size=1,
                    min_length=90,
                    max_new_tokens=100,

score: {'rouge1': 0.352112676056338, 'rouge2': 0.08571428571428572, 'rougeL': 0.18309859154929578, 'rougeLsum': 0.18309859154929578}
==========================================================================================================================
                    num_beams=5,
                    no_repeat_ngram_size=2,
                    min_length=80,
                    max_new_tokens=100,

score: {'rouge1': 0.4460431654676259, 'rouge2': 0.3065693430656934, 'rougeL': 0.4028776978417266, 'rougeLsum': 0.4028776978417266}
==========================================================================================================================
                    num_beams=3,
                    no_repeat_ngram_size=2,
                    min_length=80,
                    max_new_tokens=100,

score: {'rouge1': 0.3464566929133858, 'rouge2': 0.176, 'rougeL': 0.23622047244094488, 'rougeLsum': 0.23622047244094488}

"""

Okay, you're done.

Which model do you think produced the best summaries keeping in mind that best is in the eye of the reader?